https://towardsdatascience.com/an-introduction-to-web-scraping-with-python-a2601e8619e5


http://www.webometrics.info/es/americas/latin_america


In [4]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time

In [5]:
url = 'http://www.webometrics.info/es/americas/latin_america'
r = requests.get(url)
html_contents = r.text
html_soup = BeautifulSoup(html_contents)

In [11]:
page_url = "http://www.webometrics.info/es/americas/latin_america?page={}"
last_page = html_soup.find('li', class_='pager-last').a['href'].split('=')[1]
pages_urls = [page_url.format(i) for i in range(0, int(last_page) + 1)]
pages_urls[:2]

['http://www.webometrics.info/es/americas/latin_america?page=0',
 'http://www.webometrics.info/es/americas/latin_america?page=1']

In [7]:
# helper function to store the contents of each url to be scraped
def getAndParseURL(url):
    r = requests.get(url)
    html_soup = BeautifulSoup(r.text)
    return(html_soup)

In [8]:
ranking = []
country = []
# change to pages_urls[:] to scrape all pages (40).
for url in pages_urls[:2]:
    html_soup = getAndParseURL(url)
    tables = html_soup.find('table', class_='sticky-enabled')
    for c in tables.findAll("img", src=re.compile("logos/")):
        country_symbol = c.get('src').split('logos/')[1].split('.')[0]
        country.append(country_symbol)
    for item in tables:
        headers = []
        rows = tables.find_all('tr')
        for header in tables.find('tr').find_all('th'):
            headers.append(header.text)
    for row in tables.find_all('tr'):
        values = []
        for col in row.find_all('td'):
            values.append(col.text)
        e_dict = {headers[i]: values[i] for i in range(len(values))} 
        ranking.append(e_dict)

In [ ]:
# Create a DataFrame with the ranking
df = pd.DataFrame(ranking)
df_clean = df.dropna(how='all') # clean null values
df_clean['País'] = country # replace País(Country) column with country abbreviation
df_clean.to_csv('ranking_universities_latin_america.csv') # export to a csv file